在前面的学习中，我们从同一份样本数据中得到了三个模型，而且三个模型的拟合效果都很好，不过很显然，二次模型和三次模型都过于复杂了，属于过拟合。为了判断哪个模型的性能最优，泛化能力最强，我们将数据集划分为训练集和测试集，然后选择在测试集上表现最好的模型。使用这种方法可以有效的评估模型结果，避免过拟合。

这一节我们将学习一种避免过拟合的新方法：**正则化**（regularization）方法。

在线性回归模型 $ \hat{y} = w_0 + w_1x_1 + w_2x_2 + ... + w_nx_n = w^Tx $ 中，每一个变量 $x_i$ 就是模型的一个特征，参数 $w_i$ 代表相应的特征在模型中所占的权重。在前面的例子里，本来只有一个特征，我们人为造出了两个特征来（$x^2$ 和 $x^3$），得到了三个模型，这两个特征对模型来说其实根本没用，过拟合就是因为模型过于复杂，把这些没用的特征也学习到了。所以如果能降低这些没用特征的权重，譬如直接降到 0，那就不会出现过拟合了，为了达到这一点，我们可以稍微改进下我们的损失函数：

$$
loss = \sum_{i=1}^m (y_i - w^Tx_i)^2 + \lambda \| w \|^2
$$

这个函数相比于我们之前的平方损失函数多了一个 $\lambda \| w \|^2$，这被称之为 **正则化项** 或 **惩罚项**，其中正则化参数 $\lambda > 0$，它是一个超参数，$\| w \|^2$ 表示 $L_2$ 范数，如下：

$$
\| w \|^2 = \sum_{i=1}^m w_i^2 = w_1^2 + w_2^2 + \dots + w_m^2
$$

很显然，为了使上面的损失函数最小，模型中不相干的参数越小越好，从而降低过拟合的风险，使用这个损失函数的回归方法称为 **岭回归**（ridge regression）。

上面的 $L_2$ 范数可以改成 $L_p$ 范数，当 $p = 1$ 时，有损失函数：

$$
loss = \sum_{i=1}^m (y_i - w^Tx_i)^2 + \lambda \| w \|
$$

其中 $L_1$ 范数如下：

$$
\| w \| = \sum_{i=1}^m |w_i| = |w_1| + |w_2| + \dots + |w_m|
$$

这种回归方法称为 **LASSO 回归**（Least Absolute Shrinkage and Selection Operator）。